In [1]:
from __future__ import division, print_function, unicode_literals

In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [3]:
# reset default graph in case the notebook is rerun without restarting the kernel
tf.reset_default_graph()

In [4]:
np.random.seed(42)
tf.set_random_seed(42)

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('C:/Users/Aman Deep Singh/Documents/Python/Data/MNIST/data/')

Extracting C:/Users/Aman Deep Singh/Documents/Python/Data/MNIST/data/train-images-idx3-ubyte.gz
Extracting C:/Users/Aman Deep Singh/Documents/Python/Data/MNIST/data/train-labels-idx1-ubyte.gz
Extracting C:/Users/Aman Deep Singh/Documents/Python/Data/MNIST/data/t10k-images-idx3-ubyte.gz
Extracting C:/Users/Aman Deep Singh/Documents/Python/Data/MNIST/data/t10k-labels-idx1-ubyte.gz
